In [1]:
import pandas as pd
from influxdb_client import InfluxDBClient
from pulsar_data_collection.database_connectors.influxdb import Influxdb
import os

# Set up InfluxDB connection
token = os.getenv("DOCKER_INFLUXDB_INIT_ADMIN_TOKEN")
org = os.getenv("DOCKER_INFLUXDB_INIT_ORG")
bucket_name = os.getenv("DOCKER_INFLUXDB_INIT_BUCKET")
# token = 'mytoken'
# org = 'pulsarml'
# bucket_name = 'demo'
url = 'http://influxdb:8086'

print(token, org, bucket_name)
# Create an instance of Influxdb from your package
influxdb = Influxdb().get_database_actions()

db_login = {
    "url": url,
    "token": token,
    "org": org,
    "bucket_name": bucket_name
}

# Test InfluxDB connection
db_connection = influxdb.make_connection(**db_login)
if db_connection:
    print("Connection successful")
else:
    print("Connection failed")

mytoken pulsarml demo
Connection successful


In [2]:
# Print the existing bucket names
client = InfluxDBClient(url=url, token=token, org=org)

buckets_api = client.buckets_api()
buckets = buckets_api.find_buckets()

for bucket in buckets.buckets:
    print(bucket.name)

_tasks
_monitoring
demo


In [3]:
import sys
sys.path.append('../src/')

from gen_data_and_simulate_drift import DriftIntensity, DriftSimulator, GenerateFakeData
from training_script import Classifier

In [4]:
pokemon_test_data='pokemon.csv'
SAMPLE_SIZE=1000
target = 'Legendary'
genertor_fake_data = GenerateFakeData(path_ref_data=pokemon_test_data, sample_size=SAMPLE_SIZE, target=target)
sampled_data = genertor_fake_data.get_dataclass_sampling()

Legendary


In [5]:
# if the task is classification
pok_classifier = Classifier(df_train=sampled_data.train_data,
            num_features=sampled_data.list_num_col,
            cat_features=None,
            target=target,
            pkl_file_path=f'class_{target}_model.pkl')
pok_classifier.train()
pok_classifier.serialize()

0:	learn: 0.1323238	total: 110ms	remaining: 110ms
1:	learn: 0.0472178	total: 115ms	remaining: 0us


In [6]:
import numpy
from datetime import datetime
drift_sim_info = DriftSimulator(sampled_data, nb_cols_to_drift=1, drift_intensity=DriftIntensity.MODERATE)
# to get test_data after drifting
   
df_test_drifted = drift_sim_info.get_test_data_drifted()
df_test_drifted["timestamp"] = datetime.now()
print('info:',df_test_drifted.dtypes)
print('drift data:',df_test_drifted)
# df_test_drifted[target] = df_test_drifted[target].astype(int)

prediction = pok_classifier.predict(df_test=df_test_drifted)
prediction_int = [1 if e=='True' else 0 for e in prediction]
prediction_numpy = numpy.asarray(prediction_int)
print('prediction:',prediction_numpy)

number of columns to drift is : 1
select random column to drift ...
Drifting column Total
info: #                      int64
Total                  int64
HP                     int64
Attack                 int64
Defense                int64
Sp. Atk                int64
Sp. Def                int64
Speed                  int64
Generation             int64
Legendary               bool
timestamp     datetime64[ns]
dtype: object
drift data:        #  Total   HP  Attack  Defense  Sp. Atk  Sp. Def  Speed  Generation  \
81   466   1182  107     170      107      175      108     86           3   
639  495   1079   70      64       53      105       73     66           3   
521  422    853   72      63      120       77      108    108           3   
953  263   1341   59     155      196       80       72     66           1   
822  430    806   87     115       73      139       80    121           3   
..   ...    ...  ...     ...      ...      ...      ...    ...         ...   
294  633    8

In [7]:
!pip install pulsar_data_collection-0.0.0-py3-none-any.whl --force-reinstall

from pulsar_data_collection import config
from pulsar_data_collection.models import DataWithPrediction, PulseParameters
from pulsar_data_collection.pulse import Pulse

Processing ./pulsar_data_collection-0.0.0-py3-none-any.whl
  Attempting uninstall: pulsar-data-collection
    Found existing installation: pulsar-data-collection 0.0.0
    Uninstalling pulsar-data-collection-0.0.0:
      Successfully uninstalled pulsar-data-collection-0.0.0


In [8]:
params = PulseParameters(
    model_id="mod1",
    model_version="ver1",
    data_id="dat1",
    reference_data_storage=pokemon_test_data,
    target_name="Legendary",
    storage_engine="influxdb",
    timestamp_column_name="timestamp",
    login=db_login,
    other_labels={"timezone": "EST"},
)

In [9]:
pulse = Pulse(data=params)

In [10]:
from datetime import timezone

if not df_test_drifted.index.is_unique:
    df_test_drifted = df_test_drifted.reset_index(drop=True)

time = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")
capture_params = DataWithPrediction(
    data_points=df_test_drifted,
    predictions=pd.DataFrame(prediction_numpy, columns=["prediction"]),
    timestamp=time,
    features_names=df_test_drifted.columns.tolist(),
)

print(capture_params)

prediction_id=None timestamp=datetime.datetime(2023, 6, 9, 15, 37, 9, tzinfo=datetime.timezone.utc) predictions=     prediction
0             1
1             0
2             1
3             1
4             1
..          ...
395           1
396           0
397           0
398           1
399           1

[400 rows x 1 columns] data_points=       #  Total   HP  Attack  Defense  Sp. Atk  Sp. Def  Speed  Generation  \
0    466   1182  107     170      107      175      108     86           3   
1    495   1079   70      64       53      105       73     66           3   
2    422    853   72      63      120       77      108    108           3   
3    263   1341   59     155      196       80       72     66           1   
4    430    806   87     115       73      139       80    121           3   
..   ...    ...  ...     ...      ...      ...      ...    ...         ...   
395  633    889  109     107       70      132       84    103           4   
396  310   1068   64     100      15

In [11]:
pulse.capture_data(data=capture_params)

Written batch: ('demo', 'pulsarml', 'ns'). 

